In [ ]:
import pre_processing 
from sklearn.decomposition import LatentDirichletAllocation


if __name__ == "__main__":

    files = get_text.get_text_data()
    weighted_X, ivoc, vocab = weighted_vectorize(files)
 
    perplexity =list()
    likelihood = list()
    #for tuning 
    topic_n_range = range(2, 10)
    for topic_n in [2, 5, 10, 15, 20, 25, 30]:

        lda = LatentDirichletAllocation(evaluate_every=5,verbose=1,learning_decay= 0.7,
                                        learning_method='batch',n_components=topic_n)
        theta = lda.fit_transform(weighted_X);
        print("Log Likelihood: ", lda.score(weighted_X))
        print("Perplexity: ", lda.perplexity(weighted_X))
        perplexity.append(lda.score(weighted_X))
        likelihood.append(lda.perplexity(weighted_X))
     
    final_topic_n = 10#likelihood.index(min(likelihood)) +3
    lda = LatentDirichletAllocation(max_iter=100,evaluate_every=5,verbose=1,
                                        learning_method='batch',random_state=80,n_components=final_topic_n)
    theta = lda.fit_transform(weighted_X);
    print("Log Likelihood: ", lda.score(weighted_X))
    print("Perplexity: ", lda.perplexity(weighted_X))
        
    show_topics(lda,ivoc)
    #print_top_words(lda, list(vocab), n_top_words=10)  

    # document(chapter) - topic distribution identification 
    data_vectorized =weighted_X
    lda_output = theta
    topicnames = ["Topic" + str(i+1) for i in range(final_topic_n)]
    docnames = ["Chapter" + str(i+1) for i in range(len(files))]
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['Dominant topic'] = dominant_topic+1
    
    df_document_topic.to_html('Harry_potter_subplot.html') #this is good 
    #Counter(df_document_topic['Dominant topic'])
    # word - topic distribution identification 
    #word distribution over topic table 
    topic_words = dict()
    word_weights =dict()
    n_top_words = 100
    vocab = list(vocab)
    lda.components = lda.components_ /lda.components_.sum(axis=1)[:, np.newaxis]
    for topic, comp in enumerate(lda.components_):
       
        word_idx = np.argsort(comp)[::-1][:n_top_words]
    
        # store the words most relevant to the topic
        topic_words[topic] = [ivoc[i] for i in word_idx]
        comp = comp.tolist()
        comp.sort(reverse = True)
        word_weights[topic] = np.round(comp,2).tolist()
        #topic[]
        
    wordmap={}
    for k in range(final_topic_n):
        keys = topic_words[k]
        values = word_weights[k]
        temp = dict(zip(keys, values))
        wordmap[k] = temp

    
    #column names
    topicnames = ["Topic" + str(i+1) for i in range(final_topic_n)]    
    df = pd.DataFrame.from_dict(wordmap)
    df.columns=['Topic 1', 'Topic 2','Topic 3','Topic 4','Topic 5',
                
                'Topic 6','Topic 7','Topic 8','Topic 9', 'Topic 10']
    
    df =df.fillna(0)
    df= df.div(df.sum(axis=1), axis=0)
    df.to_html('Harry_potter_word.html')
    

    #generating graphs 
    frames = [] 
    sentence_frames = [] 
    for chapter in range(1, 200):
        evidence = {}
        d2 = {}
        for i in range(1,11):
            input_text = files
            topic_number = i
            chapternumber = chapter
            #keys = topic_words[topic_number-1]
            #wordTopic = dict(zip(keys, [topic_number]*len(keys)))
            #clean_text = clean_str(input_text[chapternumber-1])    
            
            clean = input_text[chapternumber-1].replace('Mr.', 'Mister')
            clean = clean.replace('Ms.', 'Miss')
            clean = clean.replace('Mrs.', 'Miss')
            lines = clean.split('.')
            lines = list(filter(None, lines))
    
            sentence_dist=[]
            for line in lines:
                clean = line
                words = clean.split()
                value = 0 
                for word in words:
                    if word in wordmap[topic_number-1].keys(): 
                        value += wordmap[topic_number-1][word]
                    else:
                        value = value 
                try:
                    agg_score = value/len(words)
                except: 
                    ZeroDivisionError
                sentence_dist.append(agg_score)   
                
        
            input_text ='. '.join(lines)
            # dictionary to store results (regardless of story lengths)
            # Parse text
            delim = " "
            words = [s for s in input_text.split()]  # simplest tokenization method
            merged_words = [" ".join(w) for w in merge(words, 25)]
            # Sample a sliding window of context
            delim = " "
            samples = [delim.join(s) for s in sample_window(merged_words, 20, 1)]
            
            
            score = [] 
            for sample in samples: 
                try:
                    sample = clean_text(sample)
                except:
                    sample = sample
                pnt = 0 
                words = sample.split()
                for word in words:
                    if word in wordmap[topic_number-1].keys(): 
                        pnt += wordmap[topic_number-1][word]
                    else:
                        pnt = pnt 
                try:
                    #agg_score = value/len(words)
                    agg_score = pnt/500 
                except: 
                    ZeroDivisionError
                score.append(agg_score)
            
            
            d2['samples'] = samples
            # Score sentiment using indico API
            print("\n  Submitting %i samples..." % (len(samples)))
            d2['Topic. '+str(i)] = score
            print("  ...done!")
            evidence[i] = dict(zip(lines, sentence_dist))
            
            sentence_frames.append(evidence)
        df2 = pd.DataFrame()
        for k,v in d2.items():
            df2[k] = pd.Series(v)  # keys -> columns, values -> rows
        frames.append(df2)
    

    #main()
    #frames = graph(wordmap = wordmap, files=files, topic_words = topic_words)
    for i, j in zip(frames, range(1, 200)):
        i['chapter'] = j 
        
    names = ['chp '+str(i) for i in range(1, 200)]
    subtexts = [len(i) for i in frames]
    frames_df = pd.DataFrame(list(zip(names, subtexts)))
    
    # frames refer to the subtexts in each chapter (199 chapters)
    frames_concat = pd.concat(frames) # a total of 36,412 passages 
    frames_concat.index = list(range(len(frames_concat)))
    frames_concat['length'] = frames_concat.samples.apply(nltk.word_tokenize)
    frames_concat['length'] = frames_concat.length.apply(len)
    frames_concat = frames_concat[frames_concat.length >450]
    frames_concat['total'] = frames_concat.drop(columns=['samples', 'chapter', 'length']).sum(axis=1)
    final_frames = frames_concat.iloc[:,1:].div(frames_concat.total, axis=0)
    final_frames['samples'] = frames_concat.samples
    final_frames['chapter'] = frames_concat.chapter
    final_frames.drop(columns=['samples', 'chapter', 'total']).plot(figsize = (20,8))
    new_frames = final_frames.dropna()
    new_frames.index = range(0, len(new_frames))
    new_frames.drop(columns=['samples', 'chapter', 'total']).plot(figsize = (20,8))
    #final_frames.to_excel('subtext_candidates.xlsx')              
    